In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import time

kafka_topic_name = '1234'
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
        .builder \
        .appName("Structured Streaming ") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .load()

df.printSchema()


df1 = df.selectExpr("CAST(value AS STRING)", "timestamp")



df_schema_string = "order_id INT, account_number STRING, branch STRING, transaction_code STRING"


df2 = df1 \
        .select(from_csv(col("value"), df_schema_string) \
                .alias("data"), "timestamp")


df3 = df2.select("data.*", "timestamp")


    
df3.createOrReplaceTempView("data_find");
song_find_text = spark.sql("SELECT * FROM data_find")
data_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("test7Table") \
        .start()



data_agg_write_stream.awaitTermination(1)


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



False

In [16]:
df = spark.sql("SELECT * FROM test7Table")
df.show()

+--------+-------------------+------+----------------+--------------------+
|order_id|     account_number|branch|transaction_code|           timestamp|
+--------+-------------------+------+----------------+--------------------+
|       0|02XYZXYZ10017529992|    15|              CI|2024-07-03 10:13:...|
|       1|02XYZXYZ10017529992|    15|              CI|2024-07-03 10:13:...|
|       2|02XYZXYZ10017517823|    15|              CI|2024-07-03 10:13:...|
|       3|02XYZXYZ10017517823|    15|              CI|2024-07-03 10:13:...|
|       4|02XYZXYZ10017517823|    15|              CI|2024-07-03 10:13:...|
|       5|02XYZXYZ10017519116|    15|              CI|2024-07-03 10:13:...|
|       6|02XYZXYZ10017519121|    15|              CI|2024-07-03 10:13:...|
|       7|02XYZXYZ10017520752|    15|              CI|2024-07-03 10:13:...|
|       8|02XYZXYZ10017520752|    15|              CI|2024-07-03 10:13:...|
|       9|02XYZXYZ10017520752|    15|              CI|2024-07-03 10:13:...|
|      10|02

In [23]:
df_count = df.count()
df_count

53